Runs from uq_score, experiment_sampling, synthetic_data (training), few_shot inference, and step3_score. 

## Global Cells

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --no-cache-dir --force-reinstall git+https://github.com/tlebryk/IE-UQ.git@develop

In [ ]:
!pip install --no-deps --no-cache-dir --force-reinstall "nerre[dev] @ git+https://github.com/tlebryk/NERRE.git@dev"

In [ ]:
from ie_uq import experiment_sampling, synthetic_data, few_shot_inference_extraction, uq_score
import os 
from doping import step3_score

In [ ]:
import os 
model_id = "meta-llama/Llama-3.2-3B-Instruct"
dataset_path = '/content/drive/MyDrive/nlp/Final_project/IE-UQ/runs/random-100-dataset/train_dataset.jsonl'
inference_dataset_path: str = '/content/drive/MyDrive/nlp/Final_project/IE-UQ/runs/random-100-dataset/train_dataset.jsonl'
mode="synth_json"
output_dir = "/content/drive/MyDrive/nlp/Final_project/IE-UQ/runs"
bnb_dict = {}
peft_dict = None
sft_dict = None
model_dict = None
generation_dict = None
quick_mode=False
budget = 100

synth_json_dir =  os.path.join(output_dir, "synth_json_gen")
synth_span_dir =  os.path.join(output_dir, "synth_span_gen")
extraction_dir =  os.path.join(output_dir, "synth_extraction_gen")


random_baseline_dir = os.path.join(output_dir, "random_baseline_uq_results")
active_learning_dir = os.path.join(output_dir, "active_learning_uq_results")
synth_span_dir_final = os.path.join(output_dir, "synth_span_uq_results")
small = ''
# small = '_small'

test_dataset_path = f"https://raw.githubusercontent.com/tlebryk/NERRE/refs/heads/dev/doping/data/test{small}.json"
test_dataset_path = f'/content/drive/MyDrive/nlp/Final_project/NERRE/doping/data/test{small}.json'

## random baseline 

In [ ]:
experiment_sampling.main(
    dataset_path=dataset_path,
    sampling_mode="random",
    output_dir=random_baseline_dir,
    budget=budget,
)

In [ ]:
synthetic_data.main(
    model_id=model_id,
    dataset_path=os.path.join(random_baseline_dir, "train_dataset.jsonl"),
    mode="extraction",
    output_dir=random_baseline_dir,
    bnb_dict=bnb_dict, 
    peft_dict=peft_dict,
    sft_dict=sft_dict,
    model_dict=model_dict,
    generation_dict=generation_dict
)

In [ ]:
few_shot_inference_extraction.main(
    model_id=random_baseline_dir,
    dataset_path=os.path.join(random_baseline_dir, "train_dataset.jsonl"),
    inference_dataset_path=test_dataset_path,
    mode="extraction",
    output_dir=random_baseline_dir,
    bnb_dict=bnb_dict,
    model_dict=model_dict,
    generation_dict=generation_dict,
    quick_mode=quick_mode
)

In [ ]:
step3_score.main(
    test_file=test_dataset_path,
    pred_file=os.path.join(random_baseline_dir, "fewshot2output.json"),
    plot=True,
    loud=True,
    schema_type=True,
    enforce_lowercase=True,
    output_dir=random_baseline_dir)

## Active learning

In [ ]:
# add perplexity score generation here 
uq_score.main(
    model_id = model_id,
    dataset_path = dataset_path,
    mode = "extraction",
    output_dir = active_learning_dir,
    uq_metric = "calculate_perplexity_raw",
    # full_output = True,
)

In [ ]:
experiment_sampling.main(
    dataset_path=dataset_path,
    sampling_mode="active_learning",
    output_dir=active_learning_dir,
    budget=budget,
)

In [ ]:
synthetic_data.main(
    model_id=model_id,
    dataset_path=os.path.join(active_learning_dir, "train_dataset.jsonl"),
    mode="extraction",
    output_dir=active_learning_dir,
    bnb_dict=bnb_dict, 
    peft_dict=peft_dict,
    sft_dict=sft_dict,
    model_dict=model_dict,
    generation_dict=generation_dict
)

In [ ]:
few_shot_inference_extraction.main(
    model_id=active_learning_dir,
    dataset_path=os.path.join(active_learning_dir, "train_dataset.jsonl"),
    inference_dataset_path=test_dataset_path,
    mode="extraction",
    output_dir=active_learning_dir,
    bnb_dict=bnb_dict,
    model_dict=model_dict,
    generation_dict=generation_dict,
    quick_mode=quick_mode
)

In [ ]:
step3_score.main(
    test_file=test_dataset_path,
    pred_file=os.path.join(active_learning_dir, "fewshot2output.json"),
    plot=True,
    loud=True,
    schema_type=True,
    enforce_lowercase=True,
    output_dir=active_learning_dir)

## Synthetic Data 

In [ ]:
uq_score.main(
    model_id = model_id,
    dataset_path = os.path.join(synth_span_dir, "few_shot_output.jsonl"),
    mode = "synth_span",
    output_dir = synth_span_dir_final,
    uq_metric = "calculate_perplexity_raw",
    # full_output = True,
)

In [ ]:
experiment_sampling.main(
    dataset_path = dataset_path,
    sampling_mode = "synth_span",
    output_dir = synth_span_dir_final,
    budget = 100,
    # full_output = True,
)

In [ ]:
synthetic_data.main(
    model_id = model_id,
    dataset_path = os.path.join(synth_span_dir, "train_dataset.jsonl"),
    mode = "extraction",
    output_dir = synth_span_dir_final,
    bnb_dict = bnb_dict,
    peft_dict = peft_dict,
    sft_dict = sft_dict,
    model_dict = model_dict,
    generation_dict = generation_dict,
)

In [ ]:
few_shot_inference_extraction.main(
    model_id=synth_span_dir_final,
    dataset_path=dataset_path,
    inference_dataset_path=test_dataset_path,
    mode="extraction",
    output_dir=synth_span_dir_final,
    bnb_dict=bnb_dict,
    model_dict=model_dict,
    generation_dict=generation_dict,
    quick_mode=quick_mode
)

In [ ]:
step3_score.main(
    test_file=test_dataset_path,
    pred_file=os.path.join(synth_span_dir_final, "fewshot2output.json"),
    plot=True,
    loud=True,
    schema_type=True,
    enforce_lowercase=True,
    output_dir=synth_span_dir_final)